In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
#from jh_utils.data.pandas.preprocessing import make_dummies

In [2]:
from get_df import get_data_for_ml,keep_cols

/home/joaoholz/miniconda3/envs/lestat/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from plot import lines_plot, columnX
import seaborn as sns

In [4]:
df = get_data_for_ml()
df = df[keep_cols]

/home/joaoholz/Documents/lestat/2trabalho/get_df.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df.date_time.dt.weekofyear


----
 ## Cross validation

In [5]:
n = df.shape[0]
n_train = int(n*0.7) + 1
half = (n - n_train) / 2
n_validation = int(n_train + half)

In [57]:
# ! to be easy to read again
df = get_data_for_ml()
df = df[keep_cols]

/home/joaoholz/Documents/lestat/2trabalho/get_df.py:24: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df['weekofyear'] = df.date_time.dt.weekofyear


In [58]:
df_train = df.iloc[:n_train:]
df_test = df.iloc[n_validation:,:]
df_validation = df.iloc[n_train:n_validation,:]

In [59]:
metadata = {}
metadata['original'] = df.shape
metadata['columns'] = keep_cols
metadata['train'] = df_train.shape
metadata['test'] = df_test.shape
metadata['validation'] = df_validation.shape

In [60]:
df = None

---
## Benchmark

In [61]:
from statsmodels.regression.linear_model import OLS
from metrics import evaluate

In [62]:
def transform(df):
    y = df.iloc[:,0].to_numpy() 
    x = df.iloc[:,1:].to_numpy()
    return {'x':x,'y':y}

In [63]:
train = transform(df_train)
test = transform(df_test)
validation = transform(df_validation)

In [64]:
ols_model = OLS(train['y'],train['x'])

In [65]:
ols_model_fit = ols_model.fit()
y_pred = ols_model_fit.predict(test['x'])
y_pred_val = ols_model_fit.predict(validation['x'])

In [68]:
## test
evaluate(y_pred,test['y'])

,metrics_name,metrics
0,max_error,9.462172
1,r2_score,0.867884
2,mean_squared_error,1.649207
3,median_absolute_error,0.712802
4,mean_absolute_error,0.932137
5,explained_variance_score,0.869216


In [69]:
## validation
evaluate(y_pred_val,validation['y'])

,metrics_name,metrics
0,max_error,12.252112
1,r2_score,0.151893
2,mean_squared_error,12.817205
3,median_absolute_error,1.054911
4,mean_absolute_error,2.350192
5,explained_variance_score,0.304256
